This notebook creates the input data for the second visualisation for D3, which is a line graph for the conviction rate of people charged with at least one violent crime and are convicted of at least one violent crime, and the same for sex offences.

This notebook takes the general dataset of court martials (or an edited version with just the relevant columns), after it has been cleaned with the first notebook + extra manual cleaning in Google Sheets using array formulas and the like to create columns that assess the Charge (s) column for each type of charge. It also requires that you have gone through the Violent Charges column and Sexual Offence charges column and marked in new columns whether they were found guilty of those offences. This is currently a lot of manual work, hopefully I will find a more efficient way to do all of this at some point! 
So the requirements for starting this notebook is a CSV with the following columns: 1) Year, 2) Guilty of Violent Offence, 3) Guilty of Sex offence, 4) A-Violent Charges, 5) B-Sex offence charges.

The guilty columns should be marked with 'Yes' where relevant, and the charges columns should be 'Contains' or 'Doesn't Contain'

In [2]:
import pandas as pd

file_path = 'GeneralDataset2010-2023.csv'
data = pd.read_csv(file_path)

data.head(), data.columns

(  Reference number       Rank Service Unit Trial Court Sentencing Date  \
 0              NaN  Signaller    Army  NaN  Colchester       07-Jan-10   
 1              NaN    Colonel    Army  NaN  Sennelager       08-Jan-10   
 2              NaN  Guardsman    Army  NaN  Colchester       13-Jan-10   
 3              NaN  Signaller    Army  NaN  Colchester       13-Jan-10   
 4              NaN  Guardsman    Army  NaN  Colchester       21-Jan-10   
 
    Year of Sentencing                                         Charge (s)  \
 0                2010  2 x Battery, 1 x Assault Occasioning Actual Bo...   
 1                2010  5 x Obtaining a money transfer by deception, 6...   
 2                2010                          1 x Absence Without Leave   
 3                2010                          1 x Absence Without Leave   
 4                2010           1 x Desertion, 1 x Absence Without Leave   
 
   Act charged under                      Finding  ...      G - Alcohol  \
 0       

Now need to isolate the columns you want to keep. If you are using the general dataset, this will remove a lot of columns. If you have already isolate the relevant columns then this will not really change the dataset but is important to run this code nonetheless.

In [3]:
columns_to_keep = ['Year of Sentencing', 'Guilty of violent offence', 'Guilty of sex offence', 'A - Violent charges', 'B - Sex offence charges']
filtered_data = data[columns_to_keep]

filtered_data

,Year of Sentencing,Guilty of violent offence,Guilty of sex offence,A - Violent charges,B - Sex offence charges
0,2010,Yes,No,Contains,Doesn't Contain
1,2010,NaN,No,Doesn't Contain,Doesn't Contain
2,2010,NaN,No,Doesn't Contain,Doesn't Contain
3,2010,NaN,No,Doesn't Contain,Doesn't Contain
4,2010,NaN,No,Doesn't Contain,Doesn't Contain
...,...,...,...,...,...
6066,2023,No,No,Contains,Contains
6067,2023,Yes,No,Contains,Contains
6068,2023,Yes,NaN,Contains,Doesn't Contain
6069,2023,NaN,NaN,Doesn't Contain,Doesn't Contain


The below code does the following 1) creates filtered data for all the rows where there is a violent charge, 2) creates filtered data for all the rows where there is a violent conviction, 3) repeats the process for sex offence charges and convictions, 4) creates counts for each of the filtered data categories just mentioned, 5) creates a dataframe for the violent charges and convictions by year, and the sex offence charges and convictions by year

In [4]:
violent_charges = filtered_data[filtered_data['A - Violent charges'] == 'Contains']

# Filter the rows where 'Guilty of violent offence' contains 'Yes'
violent_convictions = filtered_data[filtered_data['Guilty of violent offence'] == 'Yes']

sexoffence_charges = filtered_data[filtered_data['B - Sex offence charges'] == 'Contains']
sexoffence_convictions = filtered_data[filtered_data['Guilty of sex offence'] == 'Yes']

# Group by 'Year of Sentencing' and count the occurrences
violent_charges_counts = violent_charges.groupby('Year of Sentencing').size()
violent_convictions_counts = violent_convictions.groupby('Year of Sentencing').size()

sexoffence_charges_counts = sexoffence_charges.groupby('Year of Sentencing').size()
sexoffence_convictions_counts = sexoffence_convictions.groupby('Year of Sentencing').size()

# Combine the counts into a single DataFrame for comparison
violent_counts_by_year = pd.DataFrame({
    'Violent Charges': violent_charges_counts,
    'Violent Convictions': violent_convictions_counts
}).fillna(0)

sexoffence_counts_by_year = pd.DataFrame({
    'Sex Offence Charges': sexoffence_charges_counts,
    'Sex offence Convictions': sexoffence_convictions_counts
}).fillna(0)

print(sexoffence_counts_by_year)
print(violent_charges)

                    Violent Charges  Violent Convictions
Year of Sentencing                                      
2010                            170                   99
2011                            215                  155
2012                            184                  124
2013                            263                  152
2014                            249                  150
2015                            235                  162
2016                            239                  154
2017                            200                  117
2018                            203                  127
2019                            246                  156
2020                            181                  124
2021                            206                  109
2022                            197                  106
2023                            140                   72
                    Sex Offence Charges  Sex offence Convictions
Year of Sentencing     

The below calculates the violent conviction rate by year, by dividing the number of convictions by the the number of charges

In [28]:
violent_counts_by_year['Violent Conviction Rate'] = (violent_counts_by_year['Violent Convictions'] / violent_counts_by_year['Violent Charges']) * 100

violent_counts_by_year['Violent Conviction Rate']

Year of Sentencing
2010    58.235294
2011    72.093023
2012    67.391304
2013    57.794677
2014    60.240964
2015    68.936170
2016    64.435146
2017    58.500000
2018    62.561576
2019    63.414634
2020    68.508287
2021    52.912621
2022    53.807107
2023    51.428571
Name: Violent Conviction Rate, dtype: float64

The below calculates the sex offence conviction rate by year, by dividing the number of convictions by the the number of charges

In [37]:
sexoffence_counts_by_year['Sex Offence Conviction Rate'] = (sexoffence_counts_by_year['Sex offence Convictions'] / sexoffence_counts_by_year['Sex Offence Charges']) * 100

sexoffence_counts_by_year['Sex Offence Conviction Rate']

Year of Sentencing
2010    68.965517
2011    85.294118
2012    63.333333
2013    60.000000
2014    55.319149
2015    64.444444
2016    61.818182
2017    51.851852
2018    57.894737
2019    42.105263
2020    60.784314
2021    50.561798
2022    39.423077
2023    56.410256
Name: Sex Offence Conviction Rate, dtype: float64

The below takes the new dataframe with violent conviction rate and converts into a new dataframe with the same column name. And does the same with the sex offence conviction rate. It also resets the index for both, so that each row of the new dataframe has a new sequential unique identifier. Then the code creates a new dataframe which merges the dataframes for violent charges and sex offence charges, using the 'Year of Sentencing' as the shared identifier between them.

In [44]:
violent_conviction_rate_df = violent_counts_by_year['Violent Conviction Rate'].to_frame(name='Violent Conviction Rate').round(2)
sex_offence_conviction_rate_df = sexoffence_counts_by_year['Sex Offence Conviction Rate'].to_frame(name='Sex Offence Conviction Rate').round(2)

violent_conviction_rate_df.reset_index(inplace=True)
sex_offence_conviction_rate_df.reset_index(inplace=True)

combined_conviction_rates_df = pd.merge(violent_conviction_rate_df, sex_offence_conviction_rate_df, on='Year of Sentencing')

combined_conviction_rates_df.head()

,Year of Sentencing,Violent Conviction Rate,Sex Offence Conviction Rate
0,2010,58.24,68.97
1,2011,72.09,85.29
2,2012,67.39,63.33
3,2013,57.79,60.00
4,2014,60.24,55.32


Now, like with the second notebook, our dataframe is much more easily visualised in D3 if it is a long form, "melted" dataframe  that goes through each 'category' (here: Violent Conviction Rate and Sex Offence Conviction Rate') one by one. I melt the columns to reflect this and also rename the columns to something more simple for my Javascript code: 'year', 'category', 'value'

In [48]:
long_format_df = pd.melt(combined_conviction_rates_df, id_vars=['Year of Sentencing'],
                         value_vars=['Violent Conviction Rate', 'Sex Offence Conviction Rate'],
                         var_name='category', value_name='value')

long_format_df.rename(columns={'Year of Sentencing': 'year'}, inplace=True)


print(long_format_df.head())

   year                 category  value
0  2010  Violent Conviction Rate  58.24
1  2011  Violent Conviction Rate  72.09
2  2012  Violent Conviction Rate  67.39
3  2013  Violent Conviction Rate  57.79
4  2014  Violent Conviction Rate  60.24


Export this CSV, which is now ready to be turned into the second visualisation!

In [49]:
long_format_df.to_csv('long_format_conviction_rates.csv', index=False)
print("Data exported successfully to 'long_format_conviction_rates.csv'")

Data exported successfully to 'long_format_conviction_rates.csv'
